## 1. Install Dependencies

Install required packages that might not be available in Kaggle by default.

In [ ]:
# Check if running on Kaggle
import os
is_kaggle = os.path.exists('/kaggle')
print(f"Running on Kaggle: {is_kaggle}")

# Install any missing dependencies
if is_kaggle:
    !pip install -q pyyaml tensorboard
    print("Dependencies installed!")

## 2. Import Libraries and Setup

In [ ]:
import sys
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add project to path if running from uploaded files
if is_kaggle:
    # If you uploaded the code as a dataset, add it to path
    # sys.path.append('/kaggle/input/bethy-code')
    pass

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

## 3. Verify Dataset Path

Make sure the dataset is properly mounted.

In [ ]:
# List input datasets
if is_kaggle:
    print("Available input datasets:")
    !ls -la /kaggle/input/
    
    # Update this path based on your dataset name
    # Common paths:
    # - /kaggle/input/respiratory-sound-database/
    # - /kaggle/input/icbhi-respiratory-sound-database/
    
    print("\nChecking for audio files and annotations...")
    # Adjust this path based on your actual dataset structure
    data_paths = [
        "/kaggle/input/respiratory-sound-database",
        "/kaggle/input/icbhi-respiratory-sound-database",
        "/kaggle/input/icbhi"
    ]
    
    dataset_path = None
    for path in data_paths:
        if os.path.exists(path):
            dataset_path = path
            print(f"Found dataset at: {path}")
            !ls -la {path}
            break
    
    if dataset_path is None:
        print("⚠️ Dataset not found! Please check the path.")
else:
    dataset_path = "./data/icbhi"

## 4. Load Configuration

Load the Kaggle-specific configuration file.

In [ ]:
# Load config
config_path = 'config_kaggle.yaml' if is_kaggle else 'config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Update data path with the actual dataset location
if dataset_path:
    config['paths']['data_dir'] = dataset_path

print("Configuration loaded:")
print(f"Data directory: {config['paths']['data_dir']}")
print(f"Checkpoint directory: {config['paths']['checkpoint_dir']}")
print(f"Batch size: {config['training']['batch_size']}")
print(f"Number of epochs: {config['training']['num_epochs']}")
print(f"Device: {config['device']}")

## 5. Quick Data Exploration

Let's explore the dataset structure and verify everything is correctly loaded.

In [ ]:
from src.dataset import ICBHIDataset
from src.features import FeatureExtractor

# Initialize feature extractor
feature_extractor = FeatureExtractor(**config['features'])

# Load a small sample of the dataset
print("Loading training dataset...")
train_dataset = ICBHIDataset(
    data_dir=config['paths']['data_dir'],
    split='train',
    feature_extractor=feature_extractor,
    augment=False,
    config=config
)

print(f"\nDataset Statistics:")
print(f"Total training samples: {len(train_dataset)}")
print(f"Class names: {train_dataset.CLASS_NAMES}")
print(f"Class weights: {train_dataset.class_weights}")

# Check class distribution
labels = [sample['label'] for sample in train_dataset.samples]
unique, counts = np.unique(labels, return_counts=True)
print("\nClass distribution:")
for idx, count in zip(unique, counts):
    print(f"  {train_dataset.CLASS_NAMES[idx]}: {count} ({count/len(labels)*100:.1f}%)")

## 6. Visualize Sample Data

Visualize a sample audio file and its features.

In [ ]:
from src.visualize import plot_spectrogram, plot_class_distribution

# Get a sample
sample_idx = 0
features, label, metadata = train_dataset[sample_idx]

print(f"Sample: {metadata['filename']}")
print(f"True label: {train_dataset.CLASS_NAMES[label]}")
print(f"Features shape: {features.shape}")

# Plot mel-spectrogram
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(features[0, :128, :].numpy(), aspect='auto', origin='lower', cmap='viridis')
plt.title(f'Mel-Spectrogram: {train_dataset.CLASS_NAMES[label]}')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.colorbar()

# Plot MFCC
plt.subplot(1, 2, 2)
plt.imshow(features[0, 128:, :].numpy(), aspect='auto', origin='lower', cmap='viridis')
plt.title('MFCC')
plt.xlabel('Time')
plt.ylabel('Coefficient')
plt.colorbar()

plt.tight_layout()
plt.show()

# Plot class distribution
plot_class_distribution(
    np.array(labels),
    train_dataset.CLASS_NAMES,
    title='Training Set Class Distribution'
)

## 7. Create Model

Initialize the hybrid CNN-RNN-Attention model.

In [ ]:
from src.models import create_model

device = torch.device(config['device'] if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = create_model(config, device)
print(f"\nModel created successfully!")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

# Print model architecture
print("\nModel Architecture:")
print(model)

## 8. Training

Train the model using the training script. This will:
- Train for multiple epochs
- Validate after each epoch
- Save best model based on ICBHI score
- Apply early stopping if no improvement
- Log metrics to TensorBoard

In [ ]:
# Create output directories
os.makedirs(config['paths']['checkpoint_dir'], exist_ok=True)
os.makedirs(config['paths']['log_dir'], exist_ok=True)

# For quick testing, reduce epochs
# config['training']['num_epochs'] = 5  # Uncomment for quick test

# Run training
print("Starting training...")
print("=" * 80)

# Import and run training function
from train import train
import argparse

# Create a simple args object
class Args:
    def __init__(self):
        self.config = config_path
        self.data_dir = None
        self.checkpoint_dir = None
        self.resume = None

args = Args()
train(config, args)

## 9. Load Best Model and Evaluate

Load the best saved model and evaluate on the test set.

In [ ]:
from torch.utils.data import DataLoader
from src.metrics import MetricsCalculator
from src.visualize import plot_confusion_matrix

# Find the best model checkpoint
checkpoint_files = list(Path(config['paths']['checkpoint_dir']).glob('*_best.pth'))
if checkpoint_files:
    best_model_path = str(checkpoint_files[0])
    print(f"Loading best model from: {best_model_path}")
else:
    # Use latest checkpoint
    checkpoint_files = sorted(Path(config['paths']['checkpoint_dir']).glob('checkpoint_epoch_*.pth'))
    if checkpoint_files:
        best_model_path = str(checkpoint_files[-1])
        print(f"Loading latest checkpoint: {best_model_path}")
    else:
        print("No checkpoint found!")
        best_model_path = None

if best_model_path:
    # Load model
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    print(f"Model loaded from epoch {checkpoint['epoch']}")
    
    # Load test dataset
    test_dataset = ICBHIDataset(
        data_dir=config['paths']['data_dir'],
        split='test',
        feature_extractor=feature_extractor,
        augment=False,
        config=config
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=config['training']['batch_size'],
        shuffle=False,
        num_workers=2
    )
    
    print(f"Test dataset size: {len(test_dataset)}")
    
    # Evaluate
    print("\nEvaluating on test set...")
    all_preds = []
    all_labels = []
    all_probas = []
    
    with torch.no_grad():
        for features, labels, metadata in test_loader:
            features = features.to(device)
            logits, _ = model(features)
            probas = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_probas.extend(probas.cpu().numpy())
    
    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)
    all_probas = np.array(all_probas)
    
    # Calculate metrics
    metrics_calc = MetricsCalculator()
    metrics = metrics_calc.calculate_metrics(all_labels, all_preds, all_probas)
    
    # Print results
    metrics_calc.print_metrics(metrics, prefix="Test Set ")
    
    # Plot confusion matrix
    plot_confusion_matrix(
        metrics['confusion_matrix'],
        test_dataset.CLASS_NAMES,
        title='Test Set Confusion Matrix'
    )
    
    plot_confusion_matrix(
        metrics['confusion_matrix'],
        test_dataset.CLASS_NAMES,
        title='Test Set Confusion Matrix (Normalized)',
        normalize=True
    )

## 10. Sample Predictions with Visualization

Make predictions on some test samples and visualize the attention mechanism.

In [ ]:
from src.visualize import plot_attention_on_spectrogram, plot_prediction_confidence

# Get some test samples
num_samples = 4
fig, axes = plt.subplots(num_samples, 3, figsize=(15, num_samples*3))

for i in range(num_samples):
    features, label, metadata = test_dataset[i]
    
    # Predict
    with torch.no_grad():
        features_batch = features.unsqueeze(0).to(device)
        logits, attention_weights = model(features_batch)
        probas = torch.softmax(logits, dim=1)
        pred = torch.argmax(logits, dim=1).item()
    
    # Get data
    mel_spec = features[0, :128, :].numpy()
    attn = attention_weights[0].cpu().numpy()
    probs = probas[0].cpu().numpy()
    
    # Plot spectrogram
    axes[i, 0].imshow(mel_spec, aspect='auto', origin='lower', cmap='viridis')
    axes[i, 0].set_title(f'Sample {i+1}: {metadata["filename"][:20]}...')
    axes[i, 0].set_ylabel('Frequency')
    
    # Plot attention weights
    axes[i, 1].bar(range(len(attn)), attn, color='steelblue')
    axes[i, 1].set_title('Attention Weights')
    axes[i, 1].set_ylabel('Weight')
    
    # Plot prediction confidence
    colors = ['green' if j == pred else 'red' if j == label else 'gray' for j in range(4)]
    axes[i, 2].bar(test_dataset.CLASS_NAMES, probs, color=colors)
    axes[i, 2].set_title(f'True: {test_dataset.CLASS_NAMES[label]} | Pred: {test_dataset.CLASS_NAMES[pred]}')
    axes[i, 2].set_ylabel('Probability')
    axes[i, 2].set_ylim(0, 1)
    axes[i, 2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 11. Save Model for Submission

Save the trained model and configuration for later use or submission.

In [ ]:
# Save final model
output_path = '/kaggle/working/bethy_final_model.pth'
if best_model_path:
    import shutil
    shutil.copy(best_model_path, output_path)
    print(f"Model saved to: {output_path}")
    print(f"Model size: {os.path.getsize(output_path) / (1024*1024):.2f} MB")

# Save configuration
config_output_path = '/kaggle/working/config_final.yaml'
with open(config_output_path, 'w') as f:
    yaml.dump(config, f)
print(f"Configuration saved to: {config_output_path}")

print("\n✅ Training completed! Model ready for use.")

## Summary

This notebook demonstrated:
1. ✅ Dataset loading and exploration
2. ✅ Feature extraction (mel-spectrograms + MFCCs)
3. ✅ Model training with CNN-RNN-Attention architecture
4. ✅ Evaluation with comprehensive metrics (ICBHI Score, F1, Precision, Specificity)
5. ✅ Attention mechanism visualization
6. ✅ Model saving for deployment

### Next Steps:
- Experiment with different hyperparameters
- Try data augmentation variations
- Test on your own audio files
- Deploy as a web application